In [80]:
#getting fantasy site lines
#removing weird lines like series, half of quarter
import pandas as pd
import datetime as dt
today = dt.datetime.today()
pp_lines = pd.read_csv(f'Lines/full_lines_{today.year}_{today.month}_{today.day}.csv',index_col=0)
dog_lines = pd.read_csv(f'Lines/doglines{today.month}_{today.day}.csv',index_col=0)
pp_lines = pp_lines.loc[~pp_lines['league'].isin([250,84,251,231,192,193,227])]
unabated = pd.read_csv(f'Lines/unabated_{today.year}_{today.month}_{today.day}.csv',index_col=0)


In [81]:
unabated['date'] = pd.to_datetime(unabated['event_time'])
unabated

,player,player_id,line,stat,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,date
0,A'ja Wilson,410295,1.5,AST,7,2023-10-12T01:00:00,LVA,NYL,1,0.469388,0.530612,1.580655,2023-10-12 01:00:00
1,A'ja Wilson,410295,8.5,TRB,7,2023-10-12T01:00:00,LVA,NYL,1,0.469388,0.530612,9.214332,2023-10-12 01:00:00
2,A'ja Wilson,410295,9.0,TRB,7,2023-10-12T01:00:00,LVA,NYL,1,0.500000,0.500000,9.214332,2023-10-12 01:00:00
3,A'ja Wilson,410295,9.5,TRB,7,2023-10-12T01:00:00,LVA,NYL,9,0.451901,0.548099,9.214332,2023-10-12 01:00:00
4,A'ja Wilson,410295,21.5,PTS,7,2023-10-12T01:00:00,LVA,NYL,9,0.524889,0.475111,21.988872,2023-10-12 01:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,Yordan Alvarez,3514,0.5,Total Hits,5,2023-10-10T20:07:00,MIN,HOU,5,NaN,NaN,0.500000,2023-10-10 20:07:00
740,Zach Hrbacek,137115,38.5,Rush Yards,2,2023-10-12T01:00:00,NMS,SHS,1,0.500000,0.500000,38.680916,2023-10-12 01:00:00
741,Zach Wilson,34241,189.5,Passing Yards,1,2023-10-15T20:25:00,NYJ,PHI,1,0.500000,0.500000,192.699854,2023-10-15 20:25:00
742,Zach Wilson,34241,195.5,Passing Yards,1,2023-10-15T20:25:00,NYJ,PHI,1,0.500000,0.500000,192.699854,2023-10-15 20:25:00


In [82]:

#finding where prizepicks and underdog are different
fantasy = pp_lines.merge(dog_lines, on=['player','stat','date'],how='inner',suffixes=('_pp','_dog'))
fantasy['difference'] = ((fantasy['line_pp'] - fantasy['line_dog']))/fantasy['line_pp']
fantasy = fantasy.sort_values(by='difference')
fantasy.sample(3)

temp = fantasy.loc[(fantasy['difference'] != 0.0)]
temp = temp.loc[temp.stat != 'Hitter Fantasy Score'].sort_values(by='difference')
temp = temp[['player','line_pp','line_dog','stat', 'difference',  'date',
       'league_dog', 'team_dog' ]]
temp.to_csv(f'bets/fantasy_bets_test.csv')
temp

,player,line_pp,line_dog,stat,difference,date,league_dog,team_dog
64,Quinton Lane,27.5,32.5,Receiving Yards,-0.181818,2023-10-10,CFB,JVS
108,Brandon Ingram,11.5,13.5,PTS,-0.173913,2023-10-10,NBA,NOP
123,Walker Kessler,8.5,9.5,PTS,-0.117647,2023-10-10,NBA,UTA
38,Jerick McKinnon,8.5,9.5,Rush Yards,-0.117647,2023-10-12,NFL,KC
139,Breece Hall,59.5,65.5,Rush Yards,-0.100840,2023-10-15,NFL,NYJ
...,...,...,...,...,...,...,...,...
51,Alexy,36.5,33.5,MAPS 1-2 Kills,0.082192,2023-10-11,ESPORTS,Team SMG GC
120,Paul George,11.5,10.5,PTS,0.086957,2023-10-10,NBA,LAC
74,P.J. Washington,9.5,8.5,PTS,0.105263,2023-10-10,NBA,CHA
94,Evan Mobley,9.5,8.5,PTS,0.105263,2023-10-10,NBA,CLE


In [104]:
compare = temp.merge(unabated, on=['player','stat'],how='inner',suffixes=('','_un'))
compare

,player,line_pp,line_dog,stat,difference,date,league_dog,team_dog,player_id,line,league_id,event_time,opp,Team,count,over_prob,under_prob,pred,date_un
0,Quinton Lane,27.5,32.5,Receiving Yards,-0.181818,2023-10-10,CFB,JVS,47238,26.5,2,2023-10-10T23:30:00,JVS,LIB,2,0.494770,0.505230,28.083247,2023-10-10 23:30:00
1,Quinton Lane,27.5,32.5,Receiving Yards,-0.181818,2023-10-10,CFB,JVS,47238,27.5,2,2023-10-10T23:30:00,JVS,LIB,4,0.500000,0.500000,28.083247,2023-10-10 23:30:00
2,Quinton Lane,27.5,32.5,Receiving Yards,-0.181818,2023-10-10,CFB,JVS,47238,32.5,2,2023-10-10T23:30:00,JVS,LIB,1,0.500000,0.500000,28.083247,2023-10-10 23:30:00
3,Jerick McKinnon,8.5,9.5,Rush Yards,-0.117647,2023-10-12,NFL,KC,37298,8.5,1,2023-10-13T00:15:00,KC,DEN,6,0.507641,0.492359,9.127512,2023-10-13 00:15:00
4,Jerick McKinnon,8.5,9.5,Rush Yards,-0.117647,2023-10-12,NFL,KC,37298,9.5,1,2023-10-13T00:15:00,KC,DEN,6,0.482637,0.517363,9.127512,2023-10-13 00:15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,Javonte Williams,38.5,36.5,Rush Yards,0.051948,2023-10-12,NFL,DEN,36203,38.5,1,2023-10-13T00:15:00,KC,DEN,6,0.493252,0.506748,37.458173,2023-10-13 00:15:00
75,Dameon Pierce,54.5,51.5,Rush Yards,0.055046,2023-10-15,NFL,HOU,37925,51.5,1,2023-10-15T17:00:00,HOU,NO,1,0.500000,0.500000,53.183544,2023-10-15 17:00:00
76,Dameon Pierce,54.5,51.5,Rush Yards,0.055046,2023-10-15,NFL,HOU,37925,54.5,1,2023-10-15T17:00:00,HOU,NO,1,0.500000,0.500000,53.183544,2023-10-15 17:00:00
77,Javonte Williams,12.5,11.5,Receiving Yards,0.080000,2023-10-12,NFL,DEN,36203,11.5,1,2023-10-13T00:15:00,KC,DEN,6,0.502729,0.497271,12.106997,2023-10-13 00:15:00


In [114]:
compare = compare[['player','line_pp','line_dog','line','stat','over_prob','under_prob','date','league_dog','pred']]
compare.loc[(compare.line < compare.line_pp) & (compare.line_pp < compare.line_dog)]

,player,line_pp,line_dog,line,stat,over_prob,under_prob,date,league_dog,pred


In [115]:
compare.loc[(compare.line_pp > compare.line) & (compare.line >= compare.line_dog)]

,player,line_pp,line_dog,line,stat,over_prob,under_prob,date,league_dog,pred
46,Nate Noel,107.5,106.5,106.5,Rush Yards,0.500000,0.500000,2023-10-10,CFB,105.901789
48,Bijan Robinson,76.5,75.5,75.5,Rush Yards,0.500000,0.500000,2023-10-15,NFL,76.187282
50,Isiah Pacheco,74.5,73.5,73.5,Rush Yards,0.503523,0.496477,2023-10-12,NFL,74.855889
54,Alvin Kamara,68.5,67.5,67.5,Rush Yards,0.500000,0.500000,2023-10-15,NFL,68.185972
56,Elijah Metcalf,60.5,59.5,59.5,Receiving Yards,0.500000,0.500000,2023-10-10,CFB,60.810222
59,Garrett Wilson,55.5,54.5,54.5,Receiving Yards,0.500000,0.500000,2023-10-15,NFL,55.184078
62,Courtland Sutton,45.5,44.5,44.5,Receiving Yards,0.500493,0.499507,2023-10-12,NFL,44.414726
64,A'ja Wilson,22.0,21.5,21.5,PTS,0.524889,0.475111,2023-10-11,WNBA,21.988872
67,Skyy Moore,28.5,27.5,27.5,Receiving Yards,0.500194,0.499806,2023-10-12,NFL,28.127747
69,Terry McLaurin,56.5,54.5,54.5,Receiving Yards,0.500000,0.500000,2023-10-15,NFL,55.683959


In [ ]:

temp.loc[temp.Stat != 'Hitter Fantasy Score']

temp.loc[temp.League_dog == 'ESPORTS']

#underdog for now
fullpdf = pd.read_csv('Lines/unabated_2023_10_6.csv',index_col=0)
fullpdf.rename({'player':'Player','line':'Line','stat':'Stat'},inplace=True)
new = fullpdf.merge(pp_lines, on=['Player','Stat','Line'],how='inner').sort_values(by='o_Prob')
new.to_csv(f'bets/bets_{str(today)}.csv')
new= new.round(3)

new = new.merge(dog_lines, on=['Player','Stat'],how='inner',suffixes=('_pp','_dog'))
new['difference'] = ((new['Line_pp'] - new['Line_dog']))/new['Line_pp']
new

new.merge(temp, on=['Player','Stat','Line_pp'],how='inner')

new.columns

new[['Player', 'Stat', 'Line_pp', 'Line_dog', 'o_Prob', 'u_Prob','Date_pp','difference']].tail(20)

import yagmail
import os

attachment = os.path.abspath(f"bets/bets_{today}.csv")

send = True


if send == True:
    with open(attachment, "rb") as f:
        # send the email with the file object as an attachment

        yag = yagmail.SMTP("sharedwestover@gmail.com", oauth2_file="secrets.json")
        yag.send(subject=f"BETS! {str(today)}",attachments=f)
        
    attachment = os.path.abspath(f"bets/fantasy_bets_test.csv")